##homework1 - web acrapping 

In [1]:
from urllib.request import urlopen
from bs4 import BeautifulSoup
from tabulate import tabulate
from IPython.display import display, HTML

# specify the url
from ipython_genutils.py3compat import xrange
from tabulate import tabulate
import pandas as pd

###Ex1: Scraping the list of films Gal Gadot participated in from Wikipedia sorted by date and Presenting a readable table with the following fields per film: year, title, role and director(s).

Open Gal Gadot's Wikipedia page and Read it, using BeautifulSoup:

In [2]:
wiki = "https://en.wikipedia.org/wiki/Gal_Gadot"
page = urlopen(wiki)
soup = BeautifulSoup(page, "html.parser")

    

finding the films table:

In [3]:
right_table = soup.find('table', class_='wikitable sortable')

creating a DataFrame in order to display the films table:

In [4]:
tmp = right_table.find_all('tr')
first_row = tmp[0]
rows = tmp[1:]
headers = []

# finding the header of the table
for header in first_row.find_all('th'):
    headers.append(header.get_text())

table = [[data.get_text() for data in row.find_all('td')] for row in rows]

row_span = []

# inserting the rows with the row span value
for row_no, row_tr in enumerate(rows):
    for cell_no, content in enumerate(row_tr.find_all('td')):
        if content.has_attr("rowspan"):
            row_span.append((row_no, cell_no, int(content["rowspan"]), content.get_text()))

if row_span:
    for i in row_span:
        span = i[2]
        row = i[0]
        cell = i[1]
        text = i[3]
        # inserting the text to the right row & column
        for j in xrange(1, span):
            table[row + j].insert(cell, text)

df = pd.DataFrame(data=table, columns=headers)
df = df.ix[:, 0:4]
display(df)


   

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated


,Year,Title,Role,Director(s)
0,2009\n,Fast & Furious\n,Gisele Yashar\n,Justin Lin\n
1,2010\n,Date Night\n,Natanya\n,Shawn Levy\n
2,2010\n,Knight and Day\n,Naomi\n,James Mangold\n
3,2011\n,Fast Five\n,Gisele Yashar\n,Justin Lin\n
4,2013\n,Fast & Furious 6\n,Gisele Yashar\n,Justin Lin\n
5,2014\n,Kicking Out Shoshana\n,Mirit Ben Harush\n,Shay Kanot\n
6,2015\n,Furious 7\n,Gisele Yashar\n,James Wan\n
7,2016\n,Triple 9\n,Elena Vlaslov\n,John Hillcoat\n
8,2016\n,Batman v Superman: Dawn of Justice\n,Diana Prince / Wonder Woman\n,Zack Snyder\n
9,2016\n,Criminal\n,Jill Pope\n,Ariel Vromen\n


Question 2:

Scraping and saving the table of films Gal Gadot participated in from Wikipedia

In [5]:
import re
from collections import OrderedDict

wiki = "https://en.wikipedia.org/wiki/Gal_Gadot"
page = urlopen(wiki)
soup = BeautifulSoup(page, "html.parser")

soup = soup.find(id="Film")
movies_table = soup.findNext('table', class_='wikitable sortable')
actors_list = []



going throw all co-stars in any movie GalGadot participated in and collecting all needed information

In [6]:
# The main loop itarates all the movies, and filst the cast list of each movie
for row in movies_table.find_all("tr"):
    cells = row.find_all("td")
    if not cells:
        continue
    url = row.find(href=True)['href']
    movie_page = urlopen("https://en.wikipedia.org" + url)
    movie_soup = BeautifulSoup(movie_page, "html.parser")
    cast_tags = movie_soup.find_all("span", id=re.compile(".*cast.*", re.IGNORECASE))
    if not cast_tags:
        continue
    actors = (movie_soup.find(id=re.compile(".*cast.*", re.IGNORECASE)).parent).find_next('ul')

    # this loop, find all the actors
    # for each actor, we find the actors name and actor's page
    # for each actor page, we find the details: country of birth (country), year_of_birth and number of awards
    while actors != None:
        li_number = 0
        for actor_tag in actors.select("li"):
            li_number = li_number + 1;
            birth_country = ""
            birth_year = ""
            awards = 0
            birth_details = ""
            actor_info = ""
            # in case that there is name, with no actor's page (no extra details)
            if not actor_tag.find("a"):
                co_actor_dict = OrderedDict([("Name", re.split("as", actor_tag.text)[0]),
                                             ("Year of Birth", ""),
                                             ("Country of Birth", ""),
                                             ("No. of Awards", 0)])
                actors_list.append(co_actor_dict)
                continue
            try:
                if (actor_tag.find("a").text == "Gal Gadot"):
                    continue
            except:
                continue
            try:
                # in case that the name exist in the dictionary, we will not insert it again
                co_actor_name = actor_tag.find("a").text
                try:
                    if (df["Name"].str.contains(co_actor_name).any()):
                        continue
                except:
                    pass
                actor_page = urlopen("https://en.wikipedia.org/" + actor_tag.find("a")['href'])
                actor_page__soup = BeautifulSoup(actor_page, "html.parser")
                actor_info = pd.read_html(str(actor_page__soup.find("table", class_=re.compile(".*infobox.*"))))[0]
            except:
                pass
            try:
                # finds the actor's birth details
                birth_details = (actor_info.loc[actor_info.loc[:, 0] == "Born"].loc[:, 1]).values[0]
                actor_info = actor_info.set_index(actor_info.columns[1])

            except:
                try:
                    birth_details = (actor_info.loc[actor_info.loc[:, 0] == "Born:"].loc[:, 1]).values[0]
                except:
                    pass
                pass
            try:
                # finds the actor's birth coutry ans year of birth
                birth_country = re.split(r'[(,;)]+', birth_details)[-1]
                if all(c.isdigit() for c in birth_country):
                    birth_country = "";

                birth_year = re.findall("\d{4}", birth_details)[0].strip()
            except:
                pass
            try:
                actor_birthday = "";
                if birth_year == "":
                    actor_birthday = actor_page__soup.find(class_="bday").text;
                    birth_year = re.split("-", actor_birthday)[0];
            except:
                pass
            try:
                # finds the actor's award table at the actors page
                a = actor_page__soup.find(id=re.compile(".*Awards.*", re.IGNORECASE))
                award_table = a.findNext('table')
                while (award_table != None):
                    awards = awards + len(award_table.find_all(class_="yes table-yes2"))
                    award_table = award_table.findNext('table')
            except:
                pass
            # in case that there is no awards table at the actors page, we wiil fuk the actor's award table at the actors
            # awards page: in the awards table or at the infobox of the aeards
            if (awards == 0):
                try:
                    award_url = actor_page__soup.find_all('a',
                                                          href=re.compile(
                                                              ".*/wiki/list_of_Awards_and_nominations.*",
                                                              re.IGNORECASE))
                    awards_page = urlopen("https://en.wikipedia.org" + award_url[0].get("href"))
                    awards_soup = BeautifulSoup(awards_page, "html.parser")
                    try:
                        if (awards == 0):
                            try:
                                awards_info = pd.read_html(str(awards_soup.find("table", class_="infobox")))[0]
                                awards = (awards_info.loc[awards_info.loc[:, 0] == "Awards won"].loc[:, 1]).values[
                                    0]
                            except:
                                pass
                        if (awards == 0):
                            a = awards_soup.find(id=re.compile(".*Awards.*", re.IGNORECASE))
                            award_table = a.findNext('table')
                            while (award_table != None):
                                awards = awards + len(award_table.find_all(text="Won"))
                                if (len(award_table.find_all(text="Won")) == 0):
                                    awards = awards + len(award_table.find_all(text="Won\n"))
                                award_table = award_table.findNext('table')

                    except:
                        pass
                except:
                    pass

            co_actor_dict = OrderedDict([("Name", co_actor_name),
                                         ("Year of Birth", birth_year),
                                         ("Country of Birth", birth_country),
                                         ("No. of Awards", awards)])

            actors_list.append(co_actor_dict)
            df = pd.DataFrame(actors_list)
        try:
            p = actors.find_next('dl')
            if (li_number == 1 and len(actors.find_next('ul').select('li')) == 1):
                actors = actors.find_next('ul')
            else:
                actors = None
        except:
            actors = None

df = pd.DataFrame(actors_list)

pd.set_option('display.max_rows', 350)

display (df)

,Name,Year of Birth,Country of Birth,No. of Awards
0,Vin Diesel,1967,U.S.,9
1,Paul Walker,1973,U.S.,3
2,Michelle Rodríguez,1978,United States,7
3,Jordana Brewster,1980,Panama,1
4,John Ortiz,1969,U.S.,0
5,Laz Alonso,1974,U.S.,0
6,Jack Conley,1958,U.S.,0
7,Shea Whigham,1969,U.S.,0
8,Tego Calderón,1972,U.S.,3
9,Don Omar,1978,U.S.,20


Question 3: Create a readable table which presents the number of joint movies for each co-actor\actress with Gal Gadot, and a histogram which presents the distribution of joint movies (number of co-actors per number of joint movies).

In [7]:
import re
from collections import OrderedDict
import matplotlib.pyplot as plt

Scraping and saving the table of films Gal Gadot participated in from Wikipedia:

In [8]:
#Gal Gadot wikipedia page
wiki = "https://en.wikipedia.org/wiki/Gal_Gadot"
actor_page = urlopen(wiki)
actor_soup = BeautifulSoup(actor_page, "html.parser")
movies_list = actor_soup.find("table", class_="wikitable")


creating a Dictionary for all co-stars:

In [9]:
actors_dictionary = {}

loop that going throw over any movie Gal Gadot participate in. then, going throw every star in the movie and adding him to the dictionary if his not in it. if he does, changing the counter of this co-star.

In [10]:

# The main loop itarates all the movies, and filst the cast list of each movie
for _row in movies_list.find_all("tr"):
    cells = _row.find_all("td")
    if not cells:
        continue
    url = _row.find(href=True)['href']
    movie_page = urlopen("https://en.wikipedia.org" + url)
    movie_soup = BeautifulSoup(movie_page, "html.parser")
    tags_cast = movie_soup.find_all("span", string="Cast")
    if not tags_cast:
        continue
    cast_tag = tags_cast[-1]
    # this loop, find all the actors at the movie. For each actor, we find the actors name
    for actors in cast_tag.findNext("ul").select("li"):

        #actors with no link 
        if not actors.find("a"):
            name = re.split("as", actors.text)[0]
            if name not in actors_dictionary:
                actors_dictionary[name] = 0
            actors_dictionary[name] = actors_dictionary[name] + 1
            continue;
        # do not count Gal Gadot
        if actors.find("a").text == "Gal Gadot" :
            continue
        name = actors.find("a").text
        if actors.find("a").text not in actors_dictionary:
            actors_dictionary[name] = 0
        actors_dictionary[name] =actors_dictionary[name]+ 1





NameError: name 'movies_list' is not defined

creating and presenting the table:

In [ ]:
actorsDictionary = OrderedDict(sorted(actors_dictionary.items(), key=lambda x: x[1], reverse=True))

df = pd.Series(actorsDictionary).reset_index()
df.columns = ['Actor_Name', 'No. of Joint Movies']
display(df)

creating and presenting the histogram:

In [ ]:
historgram = df.groupby('No. of Joint Movies', as_index=False).count()
historgram.columns = ['No. of Joint Movies With Gal Gadot', 'No. of Actors']

plot = historgram.set_index('No. of Joint Movies With Gal Gadot').plot.bar()


plot.set_title("No. of films with Gal Gadot")
plt.xticks(rotation=0)
plt.yticks(range(0, 331, 30))
#display the hostogram
display(plt)